<a href="https://colab.research.google.com/github/aaayup/anomaly_detection/blob/main/code/ARIMA(normal_all_predict).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data 불러오기

In [ ]:
#importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
df=pd.read_csv('/content/normal_simulation_set.csv')
df = df.copy()
df.head()

,Unnamed: 0,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,...,heart_rate1346,event1346,heart_rate1347,event1347,heart_rate1348,event1348,heart_rate1349,event1349,heart_rate1350,event1350
0,0,2020-01-01 00:00:00,96,0,75,0,97,0,80,0,...,77,0,88,0,69,0,78,0,79,0
1,1,2020-01-01 00:10:00,73,0,79,0,84,0,84,0,...,86,0,86,0,84,0,71,0,65,0
2,2,2020-01-01 00:20:00,74,0,58,1,80,0,70,0,...,78,0,77,0,93,0,99,0,70,0
3,3,2020-01-01 00:30:00,69,0,96,0,61,0,86,0,...,72,0,80,0,75,0,74,0,78,0
4,4,2020-01-01 00:40:00,88,0,62,0,77,0,75,0,...,95,0,66,0,90,0,89,0,75,0


#반복문

In [ ]:
df_s = df[['update_time']].copy()

for i in range(1, 1001):
    df_s[f'heart_rate{i}'] = df[f'heart_rate{i}']
    df_s[f'event{i}'] = df[f'event{i}']

# 데이터를 df_train과 df_test로 분할
df_train = df_s[:1008]
df_test = df_s[1008:]


In [ ]:
for i in range(1, 1001):
    # ARIMA 모델 학습
    model = ARIMA(df_train[f'heart_rate{i}'], order=(1, 0, 3))
    result = model.fit()

    start_idx = 1008  # 테스트 데이터셋의 시작 인덱스
    end_idx = 1008 + len(df_test)  # 테스트 데이터셋의 끝 인덱스

    forecast = result.get_prediction(start=start_idx, end=end_idx, dynamic=False)

    # 신뢰구간과 예측값 할당
    forecast_ci = forecast.conf_int()

  # 데이터프레임에 신뢰구간 추가
    df_test[f'lower_{i}'] = forecast_ci.iloc[:, 0]
    df_test[f'upper_{i}'] = forecast_ci.iloc[:, 1]

    # 'pred_y{i}' 열이 이미 존재하는 경우 삭제하고 다시 추가 (insert 메서드 사용)
    if f'pred_y{i}' in df_test.columns:
        df_test.drop(columns=[f'pred_y{i}'], inplace=True)

    # 'pred_y{i}' 열을 'event{i}' 뒤에 생성
    df_test.insert(df_test.columns.get_loc(f'event{i}') + 1, f'pred_y{i}', np.where((df_test[f'heart_rate{i}'] < df_test[f'lower_{i}']) | (df_test[f'heart_rate{i}'] > df_test[f'upper_{i}']), 1, 0))

    # 'pred{i}', 'lower_{i}', 'upper_{i} 삭제
    if f'pred{i}' in df_test.columns:
        df_test.drop(columns=[f'pred{i}'], inplace = True)
    if f'lower_{i}' in df_test.columns:
        df_test.drop(columns=[f'lower_{i}'], inplace = True)
    if f'upper_{i}' in df_test.columns:
        df_test.drop(columns=[f'upper_{i}'], inplace = True)

In [ ]:
df_test

,update_time,heart_rate1,pred_y_1,event1,pred_y1,heart_rate2,pred_y_2,event2,pred_y2,heart_rate3,...,event998,pred_y998,heart_rate999,pred_y_999,event999,pred_y999,heart_rate1000,pred_y_1000,event1000,pred_y1000
1008,2020-01-08 00:00:00,85,0,0,0,82,0,0,0,94,...,0,0,85,0,0,0,91,0,0,0
1009,2020-01-08 00:10:00,95,0,0,0,72,0,0,0,77,...,0,0,86,0,0,0,86,0,0,0
1010,2020-01-08 00:20:00,86,0,0,0,74,0,0,0,85,...,0,0,75,0,0,0,74,0,0,0
1011,2020-01-08 00:30:00,66,0,0,0,78,0,0,0,75,...,0,0,93,0,0,0,60,1,0,1
1012,2020-01-08 00:40:00,65,0,0,0,84,0,0,0,74,...,0,0,71,0,0,0,86,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,2020-01-10 08:10:00,68,0,0,0,80,0,0,0,85,...,0,0,78,0,0,0,82,0,0,0
1346,2020-01-10 08:20:00,72,0,0,0,88,0,0,0,92,...,0,0,92,0,0,0,66,0,0,0
1347,2020-01-10 08:30:00,72,0,0,0,75,0,0,0,77,...,0,0,71,0,0,0,76,0,0,0
1348,2020-01-10 08:40:00,80,0,0,0,80,0,0,0,67,...,0,0,77,0,0,0,89,0,0,0


In [ ]:
from google.colab import files
df_test.to_csv('ARIMA_normal(allpredict).csv', index=False)


# CSV 파일 다운로드
files.download('ARIMA_normal(allpredict).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>